<a href="https://colab.research.google.com/github/dpapatsarouchas/PMS/blob/master/MF_online_retail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Online retail 
1. Preprocess
2. classify?
3. cluster
4. associate
5. Select attributes
6. visualize

# Imports


In [154]:
import pandas as pd
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data

In [155]:
df_ori = pd.read_excel('/content/drive/MyDrive/mfdatasets/Online Retail.xlsx')
df = df_ori.copy()
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [156]:
print('---Shape---')
print(df.shape)
print('---Info---')
print(df.info())
print('---Describe---')
print(df.describe())

---Shape---
(541909, 8)
---Info---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None
---Describe---
            Quantity      UnitPrice     CustomerID
count  541909.000000  541909.000000  406829.000000
mean        9.552250       4.611114   15287.690570
std       218.081158      96.759853    1713.600303
min    -80995.000000  -11062.060000   12346.00000

# Exploratory Data Analysis

## Describe Features

In [157]:
# InvoiceNo
print("\n==== InvoiceNo ====\n")
print(df['InvoiceNo'].describe())

# StockCode
print("\n==== StockCode ====\n")
print(df['StockCode'].describe())

# Description
print("\n==== Description ====\n")
print(df['Description'].describe())

# Quantity
print("\n==== Quantity ====\n")
print(df['Quantity'].describe())

# InvoiceDate
print("\n==== InvoiceDate ====\n")
print(df['InvoiceDate'].describe())

# UnitPrice
print("\n==== UnitPrice ====\n")
print(df['UnitPrice'].describe())

# CustomerID
print("\n==== CustomerID ====\n")
print(df['CustomerID'].describe())

# Country
print("\n==== Country ====\n")
print(df['Country'].describe())

# Unique Values
print(df.nunique())


==== InvoiceNo ====

count     541909
unique     25900
top       573585
freq        1114
Name: InvoiceNo, dtype: int64

==== StockCode ====

count     541909
unique      4070
top       85123A
freq        2313
Name: StockCode, dtype: object

==== Description ====

count                                 540455
unique                                  4223
top       WHITE HANGING HEART T-LIGHT HOLDER
freq                                    2369
Name: Description, dtype: object

==== Quantity ====

count    541909.000000
mean          9.552250
std         218.081158
min      -80995.000000
25%           1.000000
50%           3.000000
75%          10.000000
max       80995.000000
Name: Quantity, dtype: float64

==== InvoiceDate ====

count                  541909
unique                  23260
top       2011-10-31 14:41:00
freq                     1114
first     2010-12-01 08:26:00
last      2011-12-09 12:50:00
Name: InvoiceDate, dtype: object

==== UnitPrice ====

count    541909.000000
mean

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.


InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64


# Preprocessing

### Clean the data


In [158]:
# Stripping extra spaces in the description
df['Description'] = df['Description'].str.strip()

# Dropping the rows without any invoice number
df.dropna(axis = 0, subset =['InvoiceNo'], inplace = True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

# Dropping all transactions which were done on credit
# df = df[~df['InvoiceNo'].str.contains('C')]

#### Missing values

In [159]:
# Is there any missing values?
print(df.isna().sum())
# Remove Missing values
df_nonan = df.dropna()
print(df.shape)
print(df.describe())

InvoiceNo           0
StockCode           0
Description      1455
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
(541909, 8)
            Quantity      UnitPrice     CustomerID
count  541909.000000  541909.000000  406829.000000
mean        9.552250       4.611114   15287.690570
std       218.081158      96.759853    1713.600303
min    -80995.000000  -11062.060000   12346.000000
25%         1.000000       1.250000   13953.000000
50%         3.000000       2.080000   15152.000000
75%        10.000000       4.130000   16791.000000
max     80995.000000   38970.000000   18287.000000


In [160]:
# What if we remove missing CustomerID's rows?
df_cust = df[df['CustomerID'].notna()]
print(df_cust.describe())
# Do we still have missing values?
print(df_cust.isna().sum())

            Quantity      UnitPrice     CustomerID
count  406829.000000  406829.000000  406829.000000
mean       12.061303       3.460471   15287.690570
std       248.693370      69.315162    1713.600303
min    -80995.000000       0.000000   12346.000000
25%         2.000000       1.250000   13953.000000
50%         5.000000       1.950000   15152.000000
75%        12.000000       3.750000   16791.000000
max     80995.000000   38970.000000   18287.000000
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


#### Noisy Data

### DataTransformation
#### Normalization

#### Attribute Selection

#### Discretization

#### Concept Hierachy Generation

### Data Reduction

#### Data Cube Aggregation

#### Attribute Subset Selection

In [161]:
# TODO ADD fe here

#### Numerosity Reduction

#### Dimensionality Reduction

### Feature Engineering

In [162]:
# sales = unitprice*quantity
df['Sales'] = df['UnitPrice'] * df['Quantity']
# Remove outliers??
# df = df[df['Sales'] <30000]
# df = df[df['Sales'] > -30000]

# Create date, month and year columns
# df["InvoiceDate"]=pd.to_datetime(df["InvoiceDate"]).dt.date
df["InvoiceDate"]=pd.to_datetime(df['InvoiceDate'],format='%d-%m-%Y %H:%M')
df["InvoiceMonth"]=pd.DatetimeIndex(df["InvoiceDate"]).month
df["InvoiceYear"]=pd.DatetimeIndex(df["InvoiceDate"]).year

print(df.describe())

            Quantity      UnitPrice  ...   InvoiceMonth    InvoiceYear
count  541909.000000  541909.000000  ...  541909.000000  541909.000000
mean        9.552250       4.611114  ...       7.553128    2010.921609
std       218.081158      96.759853  ...       3.509055       0.268787
min    -80995.000000  -11062.060000  ...       1.000000    2010.000000
25%         1.000000       1.250000  ...       5.000000    2011.000000
50%         3.000000       2.080000  ...       8.000000    2011.000000
75%        10.000000       4.130000  ...      11.000000    2011.000000
max     80995.000000   38970.000000  ...      12.000000    2011.000000

[8 rows x 6 columns]


In [163]:
df['InvoiceMonth'].hist()
plt.show()

#### Prepare new df for RFM Analysis 

In [164]:
# Create sales rfm
rfm_sales = df.groupby('CustomerID')['Sales'].sum()
rfm_sales = rfm_sales.reset_index()
print('Sales rfm')
print(rfm_sales.head())

# Create Frequency rfm
rfm_freq = df.groupby('CustomerID')['InvoiceNo'].count()
rfm_freq = rfm_freq.reset_index()
rfm_freq.columns = ['CustomerID', 'Frequency']
print('Frequency rfm')
print(rfm_freq.head())

# Main rfm df
rfm = pd.merge(rfm_sales, rfm_freq, on='CustomerID', how='inner')

# Create recency rfm
max_date = max(df['InvoiceDate'])
df['date_difference'] = max_date - df['InvoiceDate']
rfm_p = df.groupby('CustomerID')['date_difference'].min()
rfm_p = rfm_p.reset_index()
rfm_p['date_difference'] = rfm_p['date_difference'].dt.days

# All in one
rfm = pd.merge(rfm, rfm_p, on='CustomerID', how='inner')
rfm.columns = ['CustomerID', 'Sales', 'Frequency', 'Recency']
rfm.head()

Sales rfm
   CustomerID    Sales
0     12346.0     0.00
1     12347.0  4310.00
2     12348.0  1797.24
3     12349.0  1757.55
4     12350.0   334.40
Frequency rfm
   CustomerID  Frequency
0     12346.0          2
1     12347.0        182
2     12348.0         31
3     12349.0         73
4     12350.0         17


,CustomerID,Sales,Frequency,Recency
0,12346.0,0.00,2,325
1,12347.0,4310.00,182,1
2,12348.0,1797.24,31,74
3,12349.0,1757.55,73,18
4,12350.0,334.40,17,309


##### Visualization

In [165]:
# Look at the distribution of Recency
plt.figure(figsize=(10,5))
sns.distplot(rfm["Sales"])
plt.title("Sales Distribution")
plt.xlabel("Sales")

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


Text(0.5, 0, 'Sales')

In [166]:
# Look at the distribution of Recency
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharey=True)
sns.distplot(rfm["Recency"], ax=axes[0])
axes[0].set_title("Recency Distribution")
sns.distplot(rfm["Frequency"], ax=axes[1])
axes[1].set_title("Frequency Distribution")

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


Text(0.5, 1.0, 'Frequency Distribution')

##### Check outliers

In [167]:
# Outlier Analysis of Amount Frequency and Recency

attributes = ['Sales','Frequency','Recency']
plt.rcParams['figure.figsize'] = [10,8]
sns.boxplot(data = rfm[attributes], orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
plt.title("Outliers Variable Distribution", fontsize = 14, fontweight = 'bold')
plt.ylabel("Range", fontweight = 'bold')
plt.xlabel("Attributes", fontweight = 'bold')

Text(0.5, 3.6999999999999815, 'Attributes')

In [168]:
# Removing (statistical) outliers for Sales
Q1 = rfm.Sales.quantile(0.05)
Q3 = rfm.Sales.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Sales >= Q1 - 1.5*IQR) & (rfm.Sales <= Q3 + 1.5*IQR)]

# Removing (statistical) outliers for Recency
Q1 = rfm.Recency.quantile(0.05)
Q3 = rfm.Recency.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Recency >= Q1 - 1.5*IQR) & (rfm.Recency <= Q3 + 1.5*IQR)]

# Removing (statistical) outliers for Frequency
Q1 = rfm.Frequency.quantile(0.05)
Q3 = rfm.Frequency.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Frequency >= Q1 - 1.5*IQR) & (rfm.Frequency <= Q3 + 1.5*IQR)]

# Data Visualization

In [169]:
#Monthly Sales Overview
df_Sales=df.groupby(["InvoiceMonth", "InvoiceYear"])["Sales"].sum().reset_index()
plt.figure(figsize=(15,10))
sns.barplot(x="InvoiceMonth", y="Sales", hue="InvoiceYear", data=df_Sales)
plt.title("Monthly Sales")
plt.xlabel("Month")
plt.ylabel("Sales")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Text(0, 0.5, 'Sales')

In [170]:
# visualizing the sales in the entire globe
plt.rcParams['figure.figsize'] = (15, 5)
sns.distplot(df['Sales'], color = 'crimson', hist=False)
plt.title('Distribution of Sales', fontsize = 20)
plt.xlabel('Sales worldwide')
plt.ylabel('Sales')
plt.show()

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [171]:
# visualizing the unitprice

plt.rcParams['figure.figsize'] = (15, 7)
plt.style.use('fivethirtyeight')
sns.distplot(df['UnitPrice'], color = 'lightblue', hist=False)
plt.title('Distribution of Unit price', fontsize = 20)
plt.xlabel('Different Unit Price for different items')
plt.ylabel('count')
plt.show()

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).
  warnings.warn(msg, FutureWarning)


In [172]:
# checking the different values for country in the dataset

plt.rcParams['figure.figsize'] = (12, 10)
a = df['Country'].value_counts().tail(20)
sns.barplot(x = a.values, y = a.index, palette = 'inferno')
plt.title('Bottom 20 Countries having Online Retail Market', fontsize = 20)
plt.xlabel('Names of Countries')
plt.ylabel('Count')
plt.show()

In [173]:
# checking the different values for country in the dataset

df['Country'].value_counts().head(20)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales,InvoiceMonth,InvoiceYear,date_difference
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,12,2010,373 days 04:24:00
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2010,373 days 04:24:00
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,12,2010,373 days 04:24:00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2010,373 days 04:24:00
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,12,2010,373 days 04:24:00
...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20,12,2011,0 days 00:00:00
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60,12,2011,0 days 00:00:00
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,12,2011,0 days 00:00:00
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,12,2011,0 days 00:00:00


In [174]:
# time-series plot for netherlands

dataset = df[df['Country'] == 'Netherlands']
# dataset.set_index('InvoiceDate', inplace=True)
# print(dataset)
# dataset.plot(y = 'Sales')
plt.plot_date(dataset['InvoiceDate'], dataset['Sales'], 'b-', xdate=True,)
plt.title('Time-Series for Netherlands', fontsize = 20)
plt.xlabel('Date of Purchase')
plt.ylabel('Sales Amount')
plt.show()

In [175]:
# let's look at Sales vs Invoicedate (Time series Analysis)

plt.rcParams['figure.figsize'] = (15, 5)
df.plot(x = 'InvoiceDate', y = 'Sales')
plt.title("Time Series Analysis of Sales", fontsize = 20)
plt.xlabel('Date of Purchase')
plt.ylabel('Sales')
plt.show()

/usr/local/lib/python3.7/dist-packages/pandas/plotting/_matplotlib/core.py:328: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


In [176]:
# looking at each country's sales

plt.rcParams['figure.figsize'] = (9, 12)
a = df['Sales'].groupby(df['Country']).agg('sum').sort_values(ascending = False)[1:]
# print(a)
sns.barplot(x = a.values, y = a.index, palette = 'inferno')
plt.title('Sales of all the Countries Except UK')
plt.show()

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7fe2ba5ed320> (for post_execute):


ValueError: ignored

# Association Rules

In [177]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Transactions done in France 
basket_France = (df[df['Country'] =="France"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 
  
# Transactions done in the United Kingdom 
basket_UK = (df[df['Country'] =="United Kingdom"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 
  
# Transactions done in Portugal 
basket_Por = (df[df['Country'] =="Portugal"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 
  
basket_Sweden = (df[df['Country'] =="Sweden"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1

# Encoding the datasets 
basket_encoded = basket_France.applymap(hot_encode) 
basket_France = basket_encoded 
  
basket_encoded = basket_UK.applymap(hot_encode) 
basket_UK = basket_encoded 
  
basket_encoded = basket_Por.applymap(hot_encode) 
basket_Por = basket_encoded 
  
basket_encoded = basket_Sweden.applymap(hot_encode) 
basket_Sweden = basket_encoded 

# Building the model 
frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
25,(JUMBO BAG WOODLAND ANIMALS),(POSTAGE),0.065076,0.650759,0.065076,1.000000,1.536667,0.022727,inf
185,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.086768,0.108460,0.084599,0.975000,8.989500,0.075188,35.661605
184,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER CUPS),0.086768,0.117137,0.084599,0.975000,8.323611,0.074435,35.314534
192,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.071584,0.108460,0.069414,0.969697,8.940606,0.061650,29.420824
191,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER CUPS),0.071584,0.117137,0.069414,0.969697,8.278339,0.061029,29.134490


# Clustering

## Prepare data

In [178]:
from sklearn.preprocessing import StandardScaler
# Rescaling the attributes

rfm_df = rfm[['Sales', 'Frequency', 'Recency']]

# Instantiate
scaler = StandardScaler()

# fit_transform
rfm_df_scaled = scaler.fit_transform(rfm_df)
rfm_df_scaled.shape
rfm_df_scaled = pd.DataFrame(rfm_df_scaled)
rfm_df_scaled.columns = ['Sales', 'Frequency', 'Recency']
rfm_df_scaled.head()

,Sales,Frequency,Recency
0,-0.723738,-0.752888,2.301611
1,1.731617,1.042467,-0.906466
2,0.300128,-0.463636,-0.183658
3,0.277517,-0.044720,-0.738141
4,-0.533235,-0.603275,2.143188


## K-Means

In [194]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Finding the Optimal Number of Clusters
# Elbow-curve/SSD

ssd = []
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
for num_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=num_clusters, max_iter=1000)
    kmeans.fit(rfm_df_scaled)
    
    ssd.append(kmeans.inertia_)
    
# plot the SSDs for each n_clusters
plt.plot(ssd)

In [180]:
from sklearn.metrics import silhouette_score
# Silhouette analysis
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for num_clusters in range_n_clusters:
    
    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, max_iter=1000)
    kmeans.fit(rfm_df_scaled)
    
    cluster_labels = kmeans.labels_
    
    # silhouette score
    silhouette_avg = silhouette_score(rfm_df_scaled, cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))

For n_clusters=2, the silhouette score is 0.5415858652525395
For n_clusters=3, the silhouette score is 0.5084896296141937
For n_clusters=4, the silhouette score is 0.47782241003860476
For n_clusters=5, the silhouette score is 0.46458018311256893
For n_clusters=6, the silhouette score is 0.41758431193450735
For n_clusters=7, the silhouette score is 0.41736609082715437
For n_clusters=8, the silhouette score is 0.4076109798219592


In [181]:
# Final model with k=3
kmeans = KMeans(n_clusters=5, max_iter=1000)
kmeans.fit(rfm_df_scaled)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=1000,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [182]:
kmeans.labels_

array([4, 0, 1, ..., 4, 1, 1], dtype=int32)

In [183]:
# assign the label
rfm['Cluster_Id'] = kmeans.labels_
rfm.head()

,CustomerID,Sales,Frequency,Recency,Cluster_Id
0,12346.0,0.00,2,325,4
1,12347.0,4310.00,182,1,0
2,12348.0,1797.24,31,74,1
3,12349.0,1757.55,73,18,0
4,12350.0,334.40,17,309,4


In [184]:
# Box plot to visualize Cluster Id vs Frequency

sns.boxplot(x='Cluster_Id', y='Sales', data=rfm)

In [185]:
# Box plot to visualize Cluster Id vs Frequency

sns.boxplot(x='Cluster_Id', y='Frequency', data=rfm)

In [186]:
# Box plot to visualize Cluster Id vs Recency

sns.boxplot(x='Cluster_Id', y='Recency', data=rfm)

In [187]:
rfm.groupby('Cluster_Id').mean()

,CustomerID,Sales,Frequency,Recency
Cluster_Id,,,,
0,15204.468215,2438.865026,141.689487,30.392421
1,15310.669776,624.930095,38.841884,50.235541
2,15485.510000,3801.257000,379.630000,19.325000
3,14803.452830,9056.373019,295.160377,19.113208
4,15368.658537,389.080741,25.604878,252.540488


## Hierarchical Clustering

In [188]:
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

mergings = linkage(rfm_df_scaled, method="average", metric='euclidean')
dendrogram(mergings)
plt.show()

In [189]:
# 4 clusters
cluster_labels = cut_tree(mergings, n_clusters=4).reshape(-1, )
cluster_labels

array([0, 0, 0, ..., 0, 0, 0])

In [190]:
# Assign cluster labels

rfm['Cluster_Id_Hierarchy'] = cluster_labels
rfm.head()

,CustomerID,Sales,Frequency,Recency,Cluster_Id,Cluster_Id_Hierarchy
0,12346.0,0.00,2,325,4,0
1,12347.0,4310.00,182,1,0,0
2,12348.0,1797.24,31,74,1,0
3,12349.0,1757.55,73,18,0,0
4,12350.0,334.40,17,309,4,0


In [191]:
# Plot Cluster Id vs Sales

sns.boxplot(x='Cluster_Id_Hierarchy', y='Sales', data=rfm)

In [192]:
# Plot Cluster Id vs Frequency

sns.boxplot(x='Cluster_Id_Hierarchy', y='Frequency', data=rfm)

In [193]:
# Plot Cluster Id vs Recency

sns.boxplot(x='Cluster_Id_Hierarchy', y='Recency', data=rfm)